In [1]:
import sys
sys.path.insert(0, '../../src')
import feedparser
import pandas as pd

import urllib

from postagem.Util import extract_domain, download_and_move_image, get_noticia_comercio
from lexical_analyzer_package import midia_lexical
from midia_postagem import midia_post
from Model.News import News
from Database import midia_table

from newsplease import NewsPlease
from bs4 import BeautifulSoup
import requests


import datetime
import util_midia

In [2]:
link = 'https://br.sputniknews.com/'

In [3]:
link_site = 'https://br.sputniknews.com'

In [4]:
req = requests.get(link)

In [10]:
noticias = BeautifulSoup(req.text, "html.parser").find_all('div', class_='b-main-news__first-title')
noticias += BeautifulSoup(req.text, "html.parser").find_all('div', class_='b-main-news__second m-item-2')
noticias += BeautifulSoup(req.text, "html.parser").find_all('div', class_='b-main-news__second m-item-3')
noticias += BeautifulSoup(req.text, "html.parser").find_all('div', class_='b-main-news__second m-item-4')
noticias += BeautifulSoup(req.text, "html.parser").find_all('div', class_='b-main-news__second m-item-5')
noticias += BeautifulSoup(req.text, "html.parser").find_all('div', class_='b-stories__title')

In [11]:
for noticia in noticias:
    print(noticia.find_all('a', href=True)[0]['href'])
    ref_link = noticia.find_all('a', href=True)[0]['href'] 
    print(link_site+ref_link)
    if (link_site not in ref_link):
        util_midia.social_news_from_link(link_site+ref_link)
        print(link_site+ref_link)
    else:
        util_midia.social_news_from_link(ref_link)
        print(ref_link)

/ciencia_tecnologia/2018111612689649-portugal-pequenos-satelites-espaco-base-acores/
https://br.sputniknews.com/ciencia_tecnologia/2018111612689649-portugal-pequenos-satelites-espaco-base-acores/
100% [............................................................................] 622838 / 62283812689487.png
C:\Users\admin\rss3\src\midia_postagem\images\12689487.png
["Pensando alto: Portugal vai albergar inédito 'porto espacial' para pequenos satélites"]
news_in_db: False
 -- no categories -- 
https://br.sputniknews.com/ciencia_tecnologia/2018111612689649-portugal-pequenos-satelites-espaco-base-acores/
/defesa/2018111612687255-caca-alemao-impotente-misseis-s400-russia/
https://br.sputniknews.com/defesa/2018111612687255-caca-alemao-impotente-misseis-s400-russia/
100% [..............................................................................] 99602 / 9960211811843.jpg
C:\Users\admin\rss3\src\midia_postagem\images\11811843.jpg
['Mídia: o mais novo caça alemão é impotente perante míssei

In [7]:
noticias[0]

<div class="b-main-news__second m-item-2"><a class="b-main-news__second-img" href="/defesa/2018111612687255-caca-alemao-impotente-misseis-s400-russia/"><img alt="Sistemas de defesa antimísseis S-400 da Rússia (foto do arquivo)" class="" height="450" src="https://cdnbr2.img.sputniknews.com/images/1181/18/11811847.jpg" title="Sistemas de defesa antimísseis S-400 da Rússia (foto do arquivo)" width="638"/></a><a class="b-main-news__second-title" href="/defesa/2018111612687255-caca-alemao-impotente-misseis-s400-russia/"><span>Mídia:</span> <span>o</span> <span>mais</span> <span>novo</span> <span>caça</span> <span>alemão</span> <span>é</span> <span>impotente</span> <span>perante</span> <span>mísseis</span> <span>S-400</span> <span>russos</span></a></div>